<a href="https://colab.research.google.com/github/Zamoca42/TIL/blob/main/OS/Mutex_Semaphores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 프로세스 동기화

- 데이터 정합성 이슈의 발생
  - 동시에 공유 데이터에 접근한다면 데이터 불일치가 발생할 수 있음
  - 데이터 일관성을 유지하려면 협력 프로세스간의 순차적인 실행을 보장하는 매커니즘이 필요

  ↪ Count값을 조정해보자

## 임계구역 문제와 해결안
### 생산자 - 소비자 문제

그림

- 무한버퍼
  - 버퍼의 크기에 실질적인 한계가 없음 
- 유한버퍼
  - 버퍼크기고정
    - 버퍼가 비어있으면 ➡ 소비자는 반드시 대기 
    - 모든 버퍼가 채워져 있으면 ➡ 생산자가 대기

1. 공유 메모리를 사용한 생산자 프로세스  
  (생산될 때마다 count 증가)

  ```
  item next_produced; 

  while (true) {
    /* next_produced에 item 생산 */
    while (count==BUFFER_SIZE); 
    /* 아무일도 안함*/
  
    buffer[in] = next_produced; 
    in = (in + 1) % BUFFER_SIZE;
    count++;
  }
  ```

2. 공유 메모리를 사용한 소비자 프로세스  
  (소비될 때마다 count 감소)

  ```
  item next_consumed; 
  while (true) {
    while (count==0);
    /* 아무일도 안함 */
    next_consumed = buffer[out]; 
    out = (out + 1) % BUFFER_SIZE;
    
    count--;
    /* next_consumed에 있는 item소비 */
  }
  ```

3. Count 표현 과정

- Count 초기값을 5로 가정

  ```
  count++ 를 풀어서 표현하면, 
    register1 = count
    register1 = register1 +1 
    count = register1
  ```

  ```
  count-- 를 풀어서 표현하면, 
    register2 = count
    register2 = register2 - 1 
    count = register2
  ```

  ```
  “count=5”라고 표현되기까지의 과정은,

  T0 : 생산자가 register1 = count 수행 {register1=5}   
  T1 : 생산자가 register1 = register1+1 수행 {register1=6} 
  T2 : 소비자가 register2 = count 수행 {register2=5} 
  T3 : 소비자가 register2 = register2 -1 수행 {register2=4}
  T4 : 생산자가 count = register1 수행 {count=6}
  T5 : 소비자가 count = register2 수행 {count=4}
  ```

- 프로세스가 뒤섞여서 출력
- 경쟁상태

- 임계구역 (Critical-Section)
  - 각 프로세스에는 임계 구역이라고 하는 코드 세그먼트가 존재 
    - 프로세스가 액세스하고, 업데이트하는 데이터, 적어도  
      하나의 다른 프로세스와 공유됨
  - 하나의 프로세스가 임계구역에서 실행 중일 때, 다른 프로세스들은 해당 임계 구역에 접근할 수 없음
  - 일반적으로 프로세스는 진입구역/퇴출구역, 나머지구역으로 구성


## 임계구역 문제 해결을 위한 요구조건

- 상호배제 (Mutual Exclusion)
  - 프로세스 $ P_i $ 가 임계구역에서 실행될 때, 다른 어떤 프로세스도 임계 구역에 접근할 수 없음

- 진행 (Progress)
  - 임계구역에서 실행중인 프로세스가 없고, 임계구역에 들어가고자 하는 프로세스가 존재한다면,  
  나머지 구역에서 실행 중이지 않은 프로세스들만 다음 임계구역에 진입할 프로세스를 결정하는데 참여할 수 있음.  
  이 선택은 무한정 연기할 수 없음

- 제한이 있는 대기시간 (Bounded Waiting)
  - 프로세스가 자신의 임계구역에 진입하려는 요청을 한 후부터,  
  그 요청이 허용될 때까지 다른 프로세스들이 자신들의 임계구역에 진입하도록 허용되는 횟수에 한계가 있어야 함

## 피터슨의 해결안 - 해결방법

- 두개의 프로세스가 두 개의 데이터 항목을 공유하게 하도록 하여 해결



